In [1139]:
import statistics
import queue
from collections import namedtuple

In [1140]:
Item = namedtuple("Item", ['index', 'value', 'weight', 'value_per_weight'])
items = [Item(index=0, value=1, weight=2, value_per_weight=0.5), Item(index=1, value=1, weight=2, value_per_weight=0.5), Item(index=2, value=1, weight=2, value_per_weight=0.5), Item(index=3, value=13, weight=7, value_per_weight=1.8571428571428572), Item(index=4, value=10, weight=5, value_per_weight=2.0), Item(index=5, value=10, weight=5, value_per_weight=2.0), Item(index=6, value=8, weight=3, value_per_weight=2.6666666666666665)]

In [1141]:
# Item = namedtuple("Item", ['index', 'value', 'weight'])
# items = [Item(index=0, value=45, weight=5), Item(index=1, value=48, weight=8), Item(index=2, value=35, weight=3)]

In [1142]:
Node = namedtuple("Node", ['direction', 'index', 'branch', 'value', 'weight', 'estimate', 'excluded_paths'])

In [1168]:
def get_children(node_value, capacity, node_estimate, items, branch, paths):
        
    optimistic_value = 0
    total_weight =  MAX_CAPACITY
    filtered_items = [item for item in items if item.index not in paths]
    
    for item in sorted(filtered_items, key=lambda x: x.value_per_weight, reverse=True):
        if total_weight-item.weight > 0 :
            optimistic_value += item.value
            total_weight -= item.weight  
        else:
            optimistic_value += total_weight * item.value_per_weight
            break
            
    optimistic_value = int(optimistic_value)

    
    item = items[branch]
    
    if capacity-item.weight >= 0 :
        left_node = Node('left', item.index, branch+1, node_value+item.value, capacity-item.weight, node_estimate, paths)
        right_node = Node('right', item.index, branch+1, node_value, capacity, optimistic_value, paths+[item.index]) #node_estimate-item.value) 
    else:
        left_node = Node('left', item.index, branch+1, -1, capacity-item.weight, -1, paths)
        right_node = Node('right', item.index, branch+1, node_value, capacity, optimistic_value, paths+[item.index]) #node_estimate-item.value)

    return left_node, right_node

<!-- ## Depth First Search -->

In [1188]:
with open('./data/ks_30_0', 'r') as input_data_file:
    input_data = input_data_file.read()

# parse the input
lines = input_data.split('\n')

firstLine = lines[0].split()
total_items = int(firstLine[0])
MAX_CAPACITY = int(firstLine[1])

items = []

for i in range(1, total_items+1):
    line = lines[i]
    parts = line.split()
    items.append(Item(i-1, int(parts[0]), int(parts[1]),  int(parts[0])/int(parts[1])))

min_capacity = min([item.weight for item in items])
total_items = len(items)

optimistic_value = 0
total_weight = MAX_CAPACITY
for item in sorted(items, key=lambda x: x.value_per_weight, reverse=True):
    if total_weight-item.weight > 0 :
        optimistic_value += item.value
        total_weight -= item.weight  
    else:
        optimistic_value += total_weight * item.value_per_weight
        break

optimistic_value = int(optimistic_value)

total_weight = MAX_CAPACITY
max_items=0
for item in sorted(items, key=lambda x: x.weight, reverse=False):
    if total_weight-item.weight > 0 :
        total_weight -= item.weight  
        max_items+=1
    else:
        break

print('max_capacity: ', MAX_CAPACITY)
print('min_capacity: ', min_capacity)
print('optimistic_value: ', optimistic_value)
print('total_items: ', total_items)
print('max_items: ', max_items)
## 99798

max_capacity:  100000
min_capacity:  10002
optimistic_value:  99998
total_items:  30
max_items:  9


In [1189]:
items = sorted(items, key=lambda x: x.value_per_weight, reverse=True)
items

[Item(index=0, value=90000, weight=90001, value_per_weight=0.9999888890123443),
 Item(index=1, value=89750, weight=89751, value_per_weight=0.9999888580628629),
 Item(index=3, value=89500, weight=89501, value_per_weight=0.9999888269404811),
 Item(index=5, value=89250, weight=89251, value_per_weight=0.9999887956437463),
 Item(index=7, value=89000, weight=89001, value_per_weight=0.9999887641711891),
 Item(index=9, value=88750, weight=88751, value_per_weight=0.9999887325213237),
 Item(index=11, value=88500, weight=88501, value_per_weight=0.9999887006926476),
 Item(index=13, value=88250, weight=88251, value_per_weight=0.999988668683641),
 Item(index=15, value=88000, weight=88001, value_per_weight=0.9999886364927671),
 Item(index=17, value=87750, weight=87751, value_per_weight=0.9999886041184716),
 Item(index=19, value=87500, weight=87501, value_per_weight=0.9999885715591822),
 Item(index=21, value=87250, weight=87251, value_per_weight=0.9999885388133087),
 Item(index=23, value=87000, weight

In [1190]:
max_value = 0
unexplored_nodes = []
optimal_path = []
optimal_index = None


left_node, right_node = get_children(0, MAX_CAPACITY, optimistic_value, items, 0, [])
unexplored_nodes.append(right_node)
unexplored_nodes.append(left_node)
# # unexplored_nodes = sorted(unexplored_nodes, key=lambda x: (x.branch, x.estimate))
unexplored_nodes

[Node(direction='right', index=0, branch=1, value=0, weight=100000, estimate=99998, excluded_paths=[0]),
 Node(direction='left', index=0, branch=1, value=90000, weight=9999, estimate=99998, excluded_paths=[])]

In [1191]:
loop_counter = 0
while True:

    loop_counter += 1
    
    if not unexplored_nodes:
        break
        
    explore_node = unexplored_nodes.pop()

    if explore_node.branch == len(items):
        continue
    
    left_node, right_node = get_children(explore_node.value, explore_node.weight, explore_node.estimate, items, explore_node.branch, explore_node.excluded_paths)
    
    # print('Exploring: ', explore_node)
    # print('Max Value: ', max_value)
    # print('Left Child: ', left_node)
    # print('Right Child: ', right_node)
    
    if right_node.estimate > max_value:
        unexplored_nodes.append(right_node)   
    else:     
        if right_node.value > max_value:
            max_value = right_node.value
            
            optimal_path = [1]*len(items)
            ignore_item = False
            for item in items:
                if item.index in left_node.excluded_paths or ignore_item:
                   optimal_path[item.index] = 0
    
                elif item.index == left_node.index:
                    ignore_item = True
                    
            # unexplored_nodes = [node for node in unexplored_nodes if node.estimate > max_value]
    

    if left_node.weight >= min_capacity and left_node.estimate >= max_value:
         unexplored_nodes.append(left_node)
    else:     
        if left_node.value > max_value:
            max_value = left_node.value
            
            optimal_path = [1]*len(items)
            ignore_item = False
            for item in items:
                if item.index in right_node.excluded_paths or ignore_item:
                   optimal_path[item.index] = 0
    
                elif item.index == right_node.index:
                    ignore_item = True
    
            # unexplored_nodes = [node for node in unexplored_nodes if node.estimate > max_value]
            # unexplored_nodes = sorted(unexplored_nodes, key=lambda x: x.estimate, reverse=False)
    
    # unexplored_nodes = sorted(unexplored_nodes, key=lambda x: x.branch, reverse=True)
            
    # print('Max Value: ', max_value)
    # print(unexplored_nodes)
    # print('---------------------------------------')

In [1192]:
max_value, loop_counter

(99798, 7350)

In [277]:
[1]*len(items)

[1, 1, 1, 1, 1, 1, 1]

In [ ]:
max_value = 0
counter=0

unexplored_nodes = []
explore_items = sorted(items, key=lambda x: x.weight, reverse=True)
set_root = explore_items.pop(0)
print(set_root, max_capacity, optimistic_value)

left_node = Node('left', index=-1, branch=0, value=0+set_root.value, weight=max_capacity-set_root.weight, estimate=optimistic_value)
right_node = Node('right', index=-1, branch=0, value=0, weight=max_capacity, estimate=optimistic_value-set_root.value)
unexplored_nodes.append(right_node)
unexplored_nodes.append(left_node)

while True:
    
    if not unexplored_nodes:
        break
    else:
        counter+=1
        
    explore_node = unexplored_nodes.pop()

    if explore_node.branch+1 > max_items:
        continue
    
    left_node, right_node, current_branch = get_children(explore_node.value, explore_node.weight, explore_node.estimate, explore_items, explore_node.branch+1)
    
    if (left_node.weight >= min_capacity):
         unexplored_nodes.append(left_node)
    else:     
        max_value = left_node.value if left_node.value > max_value else max_value
    
    if right_node.value+right_node.estimate > max_value and right_node.weight >= min_capacity :
        unexplored_nodes.append(right_node)   
    else:     
        max_value = right_node.value if right_node.value > max_value else max_value
    
    

max_value, counter

In [ ]:
items

In [ ]:
explore_node = unexplored_nodes.pop()

# while True:
#     if explore_node.branch+1 > total_items or explore_node.estimate < max_value:
#         max_value = explore_node.value if explore_node.value > max_value else max_value
#         if explore_node:
#             explore_node = unexplored_nodes.pop()
#     else:
#         break

optimistic_value + current_value <= max_value:

left_node, right_node, current_branch = get_children(explore_node.value, explore_node.weight, explore_node.estimate, items, explore_node.branch+1)


print('Exploring: ', explore_node)
print('Max Value: ', max_value)
print('Left Child: ', left_node)
print('Right Child: ', right_node)


if (left_node.weight >= min_capacity and current_branch != total_items) and left_node.value+left_node.estimate >= max_value:
    unexplored_nodes.append(left_node)
else:     
    max_value = left_node.value if left_node.value > max_value else max_value
    
if (right_node.weight >= min_capacity and current_branch != total_items) and right_node.value+right_node.estimate >= max_value:
    unexplored_nodes.append(right_node)   
else:     
    max_value = right_node.value if right_node.value > max_value else max_value

print('Max Value: ', max_value)
print(unexplored_nodes)
print('---------------------------------------')

In [ ]:
# Exploring:  Node(direction='left', index=3, branch=1, value=13, weight=2, estimate=21)
# Max Value:  0
# Left Child:  Node(direction='left', index=4, branch=2, value=-1, weight=-3, estimate=-1)
# Right Child:  Node(direction='right', index=4, branch=2, value=13, weight=2, estimate=11)
# Max Value:  0
# [Node(direction='right', index=3, branch=1, value=0, weight=10, estimate=8), Node(direction='right', index=4, branch=2, value=13, weight=2, estimate=11)]
# ---------------------------------------

In [ ]:
explore_node = unexplored_nodes.pop()

left_node, right_node, current_branch = get_children(explore_node.value, explore_node.weight, explore_node.estimate, items, explore_node.branch+1)
print(left_node)
print(right_node)


if current_branch != total_items:
    if left_node.weight >= 0:
        unexplored_nodes.append(left_node)
        unexplored_nodes.append(right_node)
    else:
        unexplored_nodes.append(right_node)
else:
    max_value = left_node.value if left_node.value > max_value else max_value
    
# unexplored_nodes = sorted(unexplored_nodes, key=lambda x: (x.branch, x.estimate))

print(max_value)
unexplored_nodes

In [ ]:
[Node(direction='left', index=1, branch=2, value=89750, weight=10249, estimate=9999.0)]

In [ ]:
right_node

In [ ]:
round(2.6666, 2)

In [ ]:
with open('./data/ks_30_0', 'r') as input_data_file:
    input_data = input_data_file.read()

# parse the input
lines = input_data.split('\n')

firstLine = lines[0].split()
item_count = int(firstLine[0])
capacity = int(firstLine[1])

items = []

for i in range(1, item_count+1):
    line = lines[i]
    parts = line.split()
    items.append(Item(i-1, int(parts[0]), int(parts[1]),  int(parts[0])/int(parts[1])))


total_value = 0
total_weight = capacity
for item in sorted(items, key=lambda x: x.value_per_weight, reverse=True):
    if total_weight-item.weight > 0 :
        total_value += item.value
        total_weight -= item.weight  
    else:
        total_value += total_weight * item.value_per_weight
        break


total_weight = capacity
max_items=0
for item in sorted(items, key=lambda x: x.weight, reverse=False):
    if total_weight-item.weight > 0 :
        total_weight -= item.weight  
        max_items+=1
    else:
        break

print(max_items, total_value)

In [ ]:
with open('./data/ks_30_0', 'r') as input_data_file:
    input_data = input_data_file.read()

In [ ]:
# parse the input
lines = input_data.split('\n')

firstLine = lines[0].split()
item_count = int(firstLine[0])
capacity = int(firstLine[1])

items = []

for i in range(1, item_count+1):
    line = lines[i]
    parts = line.split()
    items.append(Item(i-1, int(parts[0]), int(parts[1]),  int(parts[0])/int(parts[1])))

In [ ]:
total_value = 0
total_weight = capacity
for item in sorted(items, key=lambda x: x.value_per_weight, reverse=True):
    if total_weight-item.weight > 0 :
        total_value += item.value
        total_weight -= item.weight  
    else:
        total_value += total_weight * item.value_per_weight
        break

In [ ]:
items = sorted(items, key=lambda x: x.weight, reverse=False)
total_value

In [ ]:
total_weight = capacity
max_items=0
for item in items:
    if total_weight-item.weight > 0 :
        total_weight -= item.weight  
        counter+=1
    else:
        break

counter

In [ ]:
items = sorted(items, key=lambda x: x.value, reverse=True)

In [ ]:
for item in items:
    print(capacity

In [ ]:
items

In [ ]:
def optimistic_estimate(items, current_index, current_weight, remaining_capacity):
    optimistic_value = 0
    while current_index < len(items) and current_weight + items[current_index].weight <= remaining_capacity:
        optimistic_value += items[current_index].value
        remaining_capacity -= items[current_index].weight
        current_index += 1
    if current_index < len(items):
        optimistic_value += remaining_capacity * items[current_index].value_per_weight
    return optimistic_value

def knapsack_dfs_bb(items, max_weight, current_index, current_weight, current_value):
    if current_weight > max_weight:
        return 0
    
    if current_index >= len(items):
        return current_value
    
    optimistic_value = optimistic_estimate(items, current_index, current_weight, max_weight - current_weight)
    if optimistic_value + current_value <= max_value:
        return 0
    
    value_with_item = knapsack_dfs_bb(items, max_weight, current_index + 1,
                                       current_weight + items[current_index].weight,
                                       current_value + items[current_index].value)
    
    value_without_item = knapsack_dfs_bb(items, max_weight, current_index + 1,
                                          current_weight, current_value)
    
    return max(value_with_item, value_without_item)

In [ ]:
knapsack_dfs_bb(items, capacity, 0, 0, 0)